In [1]:
!gpustat

deeplearn.soic.iupui.edu  Mon Jul 13 18:17:23 2020
[0] GeForce GTX 1080    | 35'C,   0 % |     0 /  8119 MB |
[1] GeForce GTX 1080    | 35'C,   0 % |     0 /  8119 MB |
[2] GeForce RTX 2080 Ti | 45'C,   0 % |     0 / 11019 MB |
[3] GeForce RTX 2080 Ti | 32'C,   0 % |     0 / 11019 MB |
[4] GeForce GTX 1080    | 30'C,   0 % |     0 /  8119 MB |
[5] GeForce GTX 1080    | 30'C,   0 % |     0 /  8119 MB |
[6] GeForce RTX 2080 Ti | 33'C,   1 % |     0 / 11019 MB |
[7] GeForce RTX 2080 Ti | 31'C,   0 % |     0 / 11019 MB |


In [2]:
%load_ext autotime

import torch
import logging

# Create and configure logging module
log_format = '%(levelname)s %(asctime)s - %(message)s'
logging.basicConfig(filename = '../logs/bert_siamese_v4.logs', 
                   level = logging.INFO,
                   format = log_format,
                   filemode = 'w')
logger = logging.getLogger()
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

time: 310 ms


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from random import randrange
import torch.nn.functional as F
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
from argparse import ArgumentParser
import sys

time: 572 ms


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.get_device_name(0))

GeForce RTX 2080 Ti
time: 5.54 s


In [6]:
## Arguments

args = {
    'num_labels' : 2
    'const' : 4*768
    'batch_size' : 16
    'epochs' : 5
}

time: 308 ms


In [7]:
train_path = '../data/raw/liar_dataset/train.tsv'
test_path = '../data/raw/liar_dataset/test.tsv'
val_path = '../data/raw/liar_dataset/valid.tsv'


train_df = pd.read_csv(train_path, sep="\t", header=None)
test_df = pd.read_csv(test_path, sep="\t", header=None)
val_df = pd.read_csv(val_path, sep="\t", header=None)

# Fill nan (empty boxes) with 0
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)
val_df = val_df.fillna(0)

time: 300 ms


In [8]:
from tqdm import tqdm, trange
tqdm.pandas()
train_df[14] = train_df[1].progress_apply(lambda x: 1 if (x == 'true' or x == 'mostly-true') else 0)
test_df[14] = test_df[1].progress_apply(lambda x: 1 if (x == 'true' or x == 'mostly-true') else 0)
val_df[14] = val_df[1].progress_apply(lambda x: 1 if (x == 'true' or x == 'mostly-true') else 0)

100%|██████████| 1284/1284 [00:00<00:00, 291107.37it/s]

time: 105 ms


In [9]:
true_train_df = train_df[train_df[14] == 1]
false_train_df = train_df[train_df[14] == 0]

true_test_df = test_df[test_df[14] == 1]
false_test_df = test_df[test_df[14] == 0]

true_val_df = val_df[val_df[14] == 1]
false_val_df = val_df[val_df[14] == 0]

time: 168 ms


In [10]:
def true_text(dataframe):
    interm_dataframe = dataframe.groupby([4])[2].apply(lambda x: '%s' % ' '.join(x))
    interm_dataframe = pd.DataFrame(interm_dataframe)
    interm_dataframe.reset_index(inplace = True)
    interm_dataframe = interm_dataframe.rename(columns = {4: 4, 2: 15})
    return (interm_dataframe)

interm_true_train_data = true_text(true_train_df)
interm_true_test_data = true_text(true_test_df)
interm_true_val_data = true_text(true_val_df)

time: 633 ms


In [11]:
def false_text(dataframe):
    interm_dataframe = dataframe.groupby([4])[2].apply(lambda x: '%s' % ' '.join(x))
    interm_dataframe = pd.DataFrame(interm_dataframe)
    interm_dataframe.reset_index(inplace = True)
    interm_dataframe = interm_dataframe.rename(columns = {4: 4, 2: 16})
    return (interm_dataframe)

interm_false_train_data = false_text(false_train_df)
interm_false_test_data = false_text(false_test_df)
interm_false_val_data = false_text(false_val_df)

time: 674 ms


In [12]:
train_data = train_df.merge(interm_true_train_data, on = 4, how='left')
train_data = train_data.merge(interm_false_train_data, on = 4, how = 'left')

test_data = test_df.merge(interm_true_test_data, on = 4, how='left')
test_data = test_data.merge(interm_false_test_data, on = 4, how = 'left')

val_data = val_df.merge(interm_true_val_data, on = 4, how='left')
val_data = val_data.merge(interm_false_val_data, on = 4, how = 'left')

time: 70.9 ms


In [13]:
train_data.fillna(value = 0, inplace = True)
test_data.fillna(value = 0, inplace = True)
val_data.fillna(value = 0, inplace = True)

train_data[15] = train_data[15].astype(str)
train_data[16] = train_data[16].astype(str)

test_data[15] = test_data[15].astype(str)
test_data[16] = test_data[16].astype(str)

val_data[15] = val_data[15].astype(str)
val_data[16] = val_data[16].astype(str)

time: 372 ms


In [14]:
for i in trange(0, train_data.shape[0]):
    train_data[15][i] = str(train_data[15][i]).replace(str(train_data[2][i]), '')
    train_data[16][i] = str(train_data[16][i]).replace(str(train_data[2][i]), '')
    
for i in trange(0, test_data.shape[0]):
    test_data[15][i] = str(test_data[15][i]).replace(str(test_data[2][i]), '')
    test_data[16][i] = str(test_data[16][i]).replace(str(test_data[2][i]), '')
    
for i in trange(0, val_data.shape[0]):
    val_data[15][i] = str(val_data[15][i]).replace(str(val_data[2][i]), '')
    val_data[16][i] = str(val_data[16][i]).replace(str(val_data[2][i]), '')

  0%|          | 0/10240 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
  0%|          | 0/1267 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


time: 25min 20s


In [15]:
train = train_data.values
test = test_data.values
val = val_data.values

time: 28.9 ms


In [16]:
train_data.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,0,0,
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,0,0,
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,1,McCain opposed a requirement that the governm...,I'm the only person on this stage who has work...


time: 1.14 s


In [17]:
labels =        {'train':[train[i][1] for i in range(len(train))], 'test':[test[i][1] for i in range(len(test))], 'val':[val[i][1] for i in range(len(val))]}
statements =    {'train':[train[i][2] for i in range(len(train))], 'test':[test[i][2] for i in range(len(test))], 'val':[val[i][2] for i in range(len(val))]}
subjects =      {'train':[train[i][3] for i in range(len(train))], 'test':[test[i][3] for i in range(len(test))], 'val':[val[i][3] for i in range(len(val))]}
speakers =      {'train':[train[i][4] for i in range(len(train))], 'test':[test[i][4] for i in range(len(test))], 'val':[val[i][4] for i in range(len(val))]}
jobs =          {'train':[train[i][5] for i in range(len(train))], 'test':[test[i][5] for i in range(len(test))], 'val':[val[i][5] for i in range(len(val))]}
states =        {'train':[train[i][6] for i in range(len(train))], 'test':[test[i][6] for i in range(len(test))], 'val':[val[i][6] for i in range(len(val))]}
affiliations =  {'train':[train[i][7] for i in range(len(train))], 'test':[test[i][7] for i in range(len(test))], 'val':[val[i][7] for i in range(len(val))]}
credits =       {'train':[train[i][8:13] for i in range(len(train))], 'test':[test[i][8:13] for i in range(len(test))], 'val':[val[i][8:13] for i in range(len(val))]}
contexts =      {'train':[train[i][13] for i in range(len(train))], 'test':[test[i][13] for i in range(len(test))], 'val':[val[i][13] for i in range(len(val))]}
true_words =    {'train':[train[i][15] for i in range(len(train))], 'test':[test[i][15] for i in range(len(test))], 'val':[val[i][15] for i in range(len(val))]}
false_words =   {'train':[train[i][16] for i in range(len(train))], 'test':[test[i][16] for i in range(len(test))], 'val':[val[i][16] for i in range(len(val))]}

if num_labels == 6:

    def to_onehot(a):
        a_cat = [0]*len(a)
        for i in range(len(a)):
            if a[i]=='true':
                a_cat[i] = [1,0,0,0,0,0]
            elif a[i]=='mostly-true':
                a_cat[i] = [0,1,0,0,0,0]
            elif a[i]=='half-true':
                a_cat[i] = [0,0,1,0,0,0]
            elif a[i]=='barely-true':
                a_cat[i] = [0,0,0,1,0,0]
            elif a[i]=='false':
                a_cat[i] = [0,0,0,0,1,0]
            elif a[i]=='pants-fire':
                a_cat[i] = [0,0,0,0,0,1]
            else:
                print('Incorrect label')
        return a_cat

elif num_labels == 2:

    def to_onehot(a):
        a_cat = [0]*len(a)
        for i in range(len(a)):
            if a[i]=='true':
                a_cat[i] = [1,0]
            elif a[i]=='mostly-true':
                a_cat[i] = [1,0]
            elif a[i]=='half-true':
                a_cat[i] = [0,1]
            elif a[i]=='barely-true':
                a_cat[i] = [0,1]
            elif a[i]=='false':
                a_cat[i] = [0,1]
            elif a[i]=='pants-fire':
                a_cat[i] = [0,1]
            else:
                print('Incorrect label')
        return a_cat

else:

    print('Invalid number of labels. The number of labels should be either 2 or 6')

    sys.exit()


labels_onehot = {'train':to_onehot(labels['train']), 'test':to_onehot(labels['test']), 'val':to_onehot(labels['val'])}


# Preparing metadata
metadata = {'train':[0]*len(train), 'val':[0]*len(val), 'test':[0]*len(test)}

for i in range(len(train)):
    subject = subjects['train'][i]
    if subject == 0:
        subject = 'None'

    speaker = speakers['train'][i]
    if speaker == 0:
        speaker = 'None'

    job = jobs['train'][i]
    if job == 0:
        job = 'None'

    state = states['train'][i]
    if state == 0:
        state = 'None'

    affiliation = affiliations['train'][i]
    if affiliation == 0:
        affiliation = 'None'

    context = contexts['train'][i]
    if context == 0 :
        context = 'None'

    meta = subject + ' ' + speaker + ' ' + job + ' ' + state + ' ' + affiliation + ' ' + context

    metadata['train'][i] = meta

for i in range(len(val)):
    subject = subjects['val'][i]
    if subject == 0:
        subject = 'None'

    speaker = speakers['val'][i]
    if speaker == 0:
        speaker = 'None'

    job = jobs['val'][i]
    if job == 0:
        job = 'None'

    state = states['val'][i]
    if state == 0:
        state = 'None'

    affiliation = affiliations['val'][i]
    if affiliation == 0:
        affiliation = 'None'

    context = contexts['val'][i]
    if context == 0 :
        context = 'None'

    meta = subject + ' ' + speaker + ' ' + job + ' ' + state + ' ' + affiliation + ' ' + context

    metadata['val'][i] = meta

for i in range(len(test)):
    subject = subjects['test'][i]
    if subject == 0:
        subject = 'None'

    speaker = speakers['test'][i]
    if speaker == 0:
        speaker = 'None'

    job = jobs['test'][i]
    if job == 0:
        job = 'None'

    state = states['test'][i]
    if state == 0:
        state = 'None'

    affiliation = affiliations['test'][i]
    if affiliation == 0:
        affiliation = 'None'

    context = contexts['test'][i]
    if context == 0 :
        context = 'None'

    meta = subject + ' ' + speaker + ' ' + job + ' ' + state + ' ' + affiliation + ' ' + context

    metadata['test'][i] = meta


# Credit score calculation
credit_score = {'train':[0]*len(train), 'val':[0]*len(val), 'test':[0]*len(test)}
for i in range(len(train)):
    credit = credits['train'][i]
    if sum(credit) == 0:
        score = 0.5
    else:
        score = (credit[3]*0.2 + credit[2]*0.5 + credit[0]*0.75 + credit[1]*0.9 + credit[4]*1)/(sum(credit))
    credit_score['train'][i] = [score for i in range(args['const'])]

for i in range(len(val)):
    credit = credits['val'][i]
    if sum(credit) == 0:
        score = 0.5
    else:
        score = (credit[3]*0.2 + credit[2]*0.5 + credit[0]*0.75 + credit[1]*0.9 + credit[4]*1)/(sum(credit))
    credit_score['val'][i] = [score for i in range(args['const'])]

for i in range(len(test)):
    credit = credits['test'][i]
    if sum(credit) == 0:
        score = 0.5
    else:
        score = (credit[3]*0.2 + credit[2]*0.5 + credit[0]*0.75 + credit[1]*0.9 + credit[4]*1)/(sum(credit))
    credit_score['test'][i] = [score for i in range(args['const'])]

time: 6.36 s


In [18]:
class BertLayerNorm(nn.Module):
        def __init__(self, hidden_size, eps=1e-12):
            """Construct a layernorm module in the TF style (epsilon inside the square root).
            """
            super(BertLayerNorm, self).__init__()
            self.weight = nn.Parameter(torch.ones(hidden_size))
            self.bias = nn.Parameter(torch.zeros(hidden_size))
            self.variance_epsilo_n = eps

        def forward(self, x):
            u = x.mean(-1, keepdim=True)
            s = (x - u).pow(2).mean(-1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.variance_epsilon)
            return self.weight * x + self.bias


class BertForSequenceClassification(nn.Module):
    def __init__(self, num_labels=args['num_labels']): # Change number of labels here.
        super(BertForSequenceClassification, self).__init__()
        self.num_labels = num_labels
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size*4, num_labels)
        #self.fc1 = nn.Linear(config.hidden_size*2, 512)
        nn.init.xavier_normal_(self.classifier.weight)

    def forward_once(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        pooled_output = self.dropout(pooled_output)
        #logits = self.classifier(pooled_output)

        return pooled_output

    def forward(self, input_ids1, input_ids2, input_ids3, input_ids4, credit_sc):
        
        input_ids1 = input_ids1.to(torch.device('cuda'))
        input_ids2 = input_ids2.to(torch.device('cuda'))
        input_ids3 = input_ids3.to(torch.device('cuda'))
        input_ids4 = input_ids4.to(torch.device('cuda'))
        credit_sc = credit_sc.to(torch.device('cuda'))
        # forward pass of input 1
        output1 = self.forward_once(input_ids1, token_type_ids=None, attention_mask=None, labels=None)
        # forward pass of input 2
        output2 = self.forward_once(input_ids2, token_type_ids=None, attention_mask=None, labels=None)
        
        output3 = self.forward_once(input_ids3, token_type_ids=None, attention_mask=None, labels=None)
        
        output4 = self.forward_once(input_ids4, token_type_ids=None, attention_mask=None, labels=None)
        out = torch.cat((output1, output2, output3, output4), 1)

        out = torch.add(credit_sc, out)

        #out = self.fc1(out)
        logits = self.classifier(out)

        return logits

    def freeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = False

    def unfreeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = True


from pytorch_pretrained_bert import BertConfig

config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
        num_hidden_layers=12, num_attention_heads=12, intermediate_size=args['const'])


model = BertForSequenceClassification(args['num_labels'])
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): FusedLayerNorm(torch

time: 16 s


In [19]:
!gpustat

deeplearn.soic.iupui.edu  Mon Jul 13 18:43:18 2020
[0] GeForce GTX 1080    | 28'C,   0 % |     2 /  8119 MB |
[1] GeForce GTX 1080    | 28'C,   0 % |     2 /  8119 MB |
[2] GeForce RTX 2080 Ti | 30'C,   3 % |  1410 / 11019 MB | anbhimi(1407M)
[3] GeForce RTX 2080 Ti | 26'C,   0 % |     3 / 11019 MB |
[4] GeForce GTX 1080    | 28'C,   0 % |     2 /  8119 MB |
[5] GeForce GTX 1080    | 28'C,   0 % |     2 /  8119 MB |
[6] GeForce RTX 2080 Ti | 30'C,   0 % |     3 / 11019 MB |
[7] GeForce RTX 2080 Ti | 29'C,   0 % |     3 / 11019 MB |
time: 475 ms


In [20]:
# Loading the statements
X_train = statements['train']
y_train = labels_onehot['train']

X_val = statements['val']
y_val = labels_onehot['val']

X_train = X_train + X_val
y_train = y_train + y_val


X_test = statements['test']
y_test = labels_onehot['test']

# Loading the meta data
X_train_meta = metadata['train']
X_val_meta = metadata['val']
X_train_meta = X_train_meta + X_val_meta
X_test_meta = metadata['test']

# Loading Credit scores

X_train_credit = credit_score['train']
X_val_credit = credit_score['val']
X_train_credit = X_train_credit+X_val_credit
X_test_credit = credit_score['test']

X_train_true_words = true_words['train']+true_words['val']
X_test_true_words = true_words['test']

X_train_false_words = false_words['train']+false_words['val']
X_test_false_words = false_words['test']

max_seq_length_stat = 64
max_seq_length_words = 64
max_seq_length_meta = 32

class text_dataset(Dataset):
    def __init__(self,x_y_list, transform=None):

        self.x_y_list = x_y_list
        self.transform = transform

    def __getitem__(self,index):

        # Tokenize statements
        tokenized_review = tokenizer.tokenize(self.x_y_list[0][index])

        if len(tokenized_review) > max_seq_length_stat:
            tokenized_review = tokenized_review[:max_seq_length_stat]

        ids_review  = tokenizer.convert_tokens_to_ids(tokenized_review)

        padding = [0] * (max_seq_length_stat - len(ids_review))

        ids_review += padding

        assert len(ids_review) == max_seq_length_stat

        #print(ids_review)
        ids_review = torch.tensor(ids_review)

        # Tokenize metadata

        tokenized_review_meta = tokenizer.tokenize(self.x_y_list[1][index])

        if len(tokenized_review_meta) > max_seq_length_meta:
            tokenized_review_meta = tokenized_review_meta[:max_seq_length_meta]

        ids_review_meta = tokenizer.convert_tokens_to_ids(tokenized_review_meta)

        padding = [0] * (max_seq_length_meta - len(ids_review_meta))

        ids_review_meta += padding

        assert len(ids_review_meta) == max_seq_length_meta

        #print(ids_review)
        ids_review_meta = torch.tensor(ids_review_meta)

        fakeness = self.x_y_list[5][index] # color
        list_of_labels = [torch.from_numpy(np.array(fakeness))]
        
        # Tokenize True Words
        tokenized_true_words = tokenizer.tokenize(self.x_y_list[2][index])
        
        if len(tokenized_true_words) > max_seq_length_words:
            tokenized_true_words = tokenized_true_words[:max_seq_length_words]
            
        ids_true_words = tokenizer.convert_tokens_to_ids(tokenized_true_words)
        padding = [0]*(max_seq_length_words - len(ids_true_words))
        ids_true_words += padding
        assert len(ids_true_words) == max_seq_length_words
        ids_true_words = torch.tensor(ids_true_words)
        
        # Tokenize False Words
        tokenized_false_words = tokenizer.tokenize(self.x_y_list[3][index])
        
        if len(tokenized_false_words) > max_seq_length_words:
            tokenized_false_words = tokenized_false_words[:max_seq_length_words]
            
        ids_false_words = tokenizer.convert_tokens_to_ids(tokenized_false_words)
        padding = [0]*(max_seq_length_words - len(ids_false_words))
        ids_false_words += padding
        assert len(ids_false_words) == max_seq_length_words
        ids_false_words = torch.tensor(ids_false_words)
        
        credit_scr = self.x_y_list[4][index] # Credit score
        credit_scr = torch.tensor(credit_scr)
        
        return [ids_review, ids_review_meta, ids_true_words, ids_false_words, credit_scr], list_of_labels[0]

    def __len__(self):
        return len(self.x_y_list[0])

time: 1.33 s


In [21]:
# Train Statements and Justifications
train_lists = [X_train, X_train_meta, X_train_true_words, X_train_false_words, X_train_credit, y_train]

# Test Statements and Justifications
test_lists = [X_test, X_train_meta, X_test_true_words, X_test_false_words, X_test_credit, y_test]

# Preparing the data (Tokenize)
training_dataset = text_dataset(x_y_list = train_lists)
test_dataset = text_dataset(x_y_list = test_lists)

# Prepare the training dictionaries
dataloaders_dict = {'train': torch.utils.data.DataLoader(training_dataset, batch_size=args['batch_size'], shuffle=True, num_workers=0),
                   'val':torch.utils.data.DataLoader(test_dataset, batch_size=args['batch_size'], shuffle=False, num_workers=0)
                   }
dataset_sizes = {'train':len(train_lists[0]),
                'val':len(test_lists[0])}

time: 343 ms


In [22]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

logger.info('Training begins')
train_acc = []
val_acc = []
train_loss = []
val_loss = []

outputs_list = []
fakeness_list = []

def train_model(model, criterion, optimizer, scheduler, num_epochs=4):
    since = time.time()
    print('starting')
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 100
    best_acc = 0

    for epoch in range(num_epochs):
        logger.info('Training the epoch : {}'.format(epoch+1))
        logger.debug('Training stopped at epoch : {}'.format(epoch))
        epoch_start = time.time()
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0

            fakeness_corrects = 0
            
            # Iterate over data.
            for inputs, fakeness in dataloaders_dict[phase]:
                
                inputs1 = inputs[0] # News statement input
                inputs2 = inputs[1] # Meta data input
                inputs3 = inputs[2] # True Words
                inputs4 = inputs[3] # False Words
                inputs5 = inputs[4] # Credit scores input
                
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    #print(inputs)
                    outputs = model(inputs1, inputs2, inputs3, inputs4, inputs5)

                    outputs = F.softmax(outputs,dim=1)
                    outputs_list.append(outputs)
                    
                    fakeness = fakeness.to(torch.device('cuda'))
                    fakeness_list.append(fakeness)
                    loss = criterion(outputs, torch.max(fakeness.float(), 1)[1])
                    # backward + optimize only if in training phase
                    if phase == 'train':

                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs1.size(0)

                fakeness_corrects += torch.sum(torch.max(outputs, 1)[1] == torch.max(fakeness, 1)[1])

            epoch_loss = running_loss / dataset_sizes[phase]

            fakeness_acc = fakeness_corrects.double() / dataset_sizes[phase]

            print('{} total loss: {:.4f} '.format(phase,epoch_loss ))
            print('{} fakeness_acc: {:.4f}'.format(
                phase, fakeness_acc))

            # Saving training acc and loss for each epoch
            fakeness_acc1 = fakeness_acc.data
            fakeness_acc1 = fakeness_acc1.cpu()
            fakeness_acc1 = fakeness_acc1.numpy()
            train_acc.append(fakeness_acc1)
            train_loss.append(epoch_loss)

            if phase == 'val' and fakeness_acc > best_acc:
                print('Saving with accuracy of {}'.format(fakeness_acc),
                      'improved over previous {}'.format(best_acc))
                best_acc = fakeness_acc

                # Saving val acc and loss for each epoch
                fakeness_acc1 = fakeness_acc.data
                fakeness_acc1 = fakeness_acc1.cpu()
                fakeness_acc1 = fakeness_acc1.numpy()
                val_acc.append(fakeness_acc1)
                val_loss.append(epoch_loss)

                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), 'bert_model_test_noFC1_triBERT_binary_focalloss.pth')

        print('Time taken for epoch'+ str(epoch+1)+ ' is ' + str((time.time() - epoch_start)/60) + ' minutes')
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(float(best_acc)))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_acc, val_acc, train_loss, val_loss

lrlast = .0001
lrmain = .00001
optim1 = optim.Adam(
    [
        {"params":model.bert.parameters(),"lr": lrmain},
        {"params":model.classifier.parameters(), "lr": lrlast},

   ])

# Observe that all parameters are being optimized
optimizer_ft = optim1
criterion = nn.CrossEntropyLoss()

'''import focal_loss
loss_args = {"alpha": 0.5, "gamma": 2.0}
criterion = focal_loss.FocalLoss(*loss_args)'''

lambda1 = lambda epoch: epoch // 30
lambda2 = lambda epoch: 0.95 ** epoch

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 2, gamma = 0.1)

model_ft1, train_acc, val_acc, train_loss, val_loss = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=args['epochs'])

starting
Epoch 1/5
----------


/home/anbhimi/.local/lib/python3.5/site-packages/torch/optim/lr_scheduler.py:82: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train total loss: 0.5268 
train fakeness_acc: 0.7680
val total loss: 0.4478 
val fakeness_acc: 0.8508
Saving with accuracy of 0.850828729281768 improved over previous 0
Time taken for epoch1 is 10.513406018416086 minutes

Epoch 2/5
----------
train total loss: 0.4916 
train fakeness_acc: 0.8056
val total loss: 0.4446 
val fakeness_acc: 0.8564
Saving with accuracy of 0.856353591160221 improved over previous 0.850828729281768
Time taken for epoch2 is 10.529628002643586 minutes

Epoch 3/5
----------
train total loss: 0.4847 
train fakeness_acc: 0.8187
val total loss: 0.4448 
val fakeness_acc: 0.8571
Saving with accuracy of 0.8571428571428571 improved over previous 0.856353591160221
Time taken for epoch3 is 10.54173267285029 minutes

Epoch 4/5
----------
train total loss: 0.4773 
train fakeness_acc: 0.8270
val total loss: 0.4450 
val fakeness_acc: 0.8571
Time taken for epoch4 is 10.44638971487681 minutes

Epoch 5/5
----------
train total loss: 0.4781 
train fakeness_acc: 0.8264
val total l

In [23]:
refined_output_list = []

for _list in outputs_list:
    refined_output_list.extend(_list[:,0].tolist())

time: 336 ms


In [24]:
refined_output_list = np.array(refined_output_list)
refined_output_list = np.where(refined_output_list > 0.5, 1, 0)

time: 669 ms


In [25]:
refined_fakeness_list = []

for _list in fakeness_list:
    refined_fakeness_list.extend(_list[:,0].tolist())

time: 497 ms


In [26]:
from sklearn.metrics import classification_report

print (classification_report(refined_output_list, refined_fakeness_list))

              precision    recall  f1-score   support

           0       0.93      0.81      0.87     47290
           1       0.61      0.82      0.70     16665

    accuracy                           0.81     63955
   macro avg       0.77      0.82      0.78     63955
weighted avg       0.84      0.81      0.82     63955

time: 599 ms
